# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [8]:
!cat publications.tsv

pub_date	title	venue	excerpt	citation	url_slug	paper_url	slides_url
2025-09-30	Toward Quantum-Enabled Biomarker Discovery: An Outlook from Q4Bio	arXiv (quant-ph)	Outlook on hybrid quantum–classical pipelines for biomarker discovery; defines empirical quantum advantage and sketches a co-designed path to near-term feasibility.	Shah, D., Teo, M., Robinett, R. A., Madejski, S., Morrell, Z., Ramesh, S., Campbell, C., Thotakura, B., Omole, V., Hall, B., Harrow, A. W., Tomesh, T., Pearson, A. T., Chong, F. T., & Riesenfeld, S. J. (2025). "Toward Quantum-Enabled Biomarker Discovery: An Outlook from Q4Bio." <i>arXiv</i>, arXiv:2509.25904.	q4bio-quantum-enabled-biomarker-discovery-outlook	https://arxiv.org/pdf/2509.25904	
2025-08-18	Noisy Quantum Simulation Using Tracking, Uncomputation and Sampling	arXiv (quant-ph)	Introduces TUSQ, a simulator that reduces overhead for noisy circuit emulation via tracking, uncomputation, and tree-based sampling.	Dangwal, S., Oberoi, T., Sailopal, A., Shah, D., 

## Import pandas

We are using the very handy pandas library for dataframes.

In [9]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [10]:
publications = pd.read_csv("publications.tsv", sep="\t", header=0)
publications


,pub_date,title,venue,excerpt,citation,url_slug,paper_url,slides_url
0,2025-09-30,Toward Quantum-Enabled Biomarker Discovery: An...,arXiv (quant-ph),Outlook on hybrid quantum–classical pipelines ...,"Shah, D., Teo, M., Robinett, R. A., Madejski, ...",q4bio-quantum-enabled-biomarker-discovery-outlook,https://arxiv.org/pdf/2509.25904,NaN
1,2025-08-18,"Noisy Quantum Simulation Using Tracking, Uncom...",arXiv (quant-ph),"Introduces TUSQ, a simulator that reduces over...","Dangwal, S., Oberoi, T., Sailopal, A., Shah, D...",tusq-noisy-quantum-simulation-tracking-uncompu...,https://arxiv.org/pdf/2508.04880.pdf,NaN
2,2024-09-20,Circuit decompositions and scheduling for neut...,IEEE International Conference on Quantum Compu...,Compiler pipeline and scheduling that minimize...,"Nottingham, N., Perlin, M. A., Shah, D., White...",neutral-atom-compiler-decompositions-schedulin...,https://arxiv.org/pdf/2307.14996.pdf,NaN
3,2024-02-20,YFlows: Systematic Dataflow Exploration and Co...,Proceedings of the 33rd ACM SIGPLAN Internatio...,Explores dataflow choices and code generation ...,"Zhou, C., Hassman, Z., Shah, D., Richard, V., ...",yflows-simd-dataflow-cpu-inference-cc2024,https://arxiv.org/pdf/2310.00574.pdf,NaN
4,2023-08-18,Applicability of Clay/Organic Clay to Environm...,Applied Sciences,Overview of clay and organoclay materials for ...,"Qi, J., Yu, J., Shah, K. J., Shah, D. D., & Yo...",applicability-of-clay-organic-clay-to-environm...,https://www.mdpi.com/2076-3417/13/16/9395/pdf,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [11]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [12]:
import os
for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: /publication/""" + html_filename
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.slides_url)) > 5:
        md += "\nslidesurl: '" + item.slides_url + "'"

    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"

    if len(str(item.slides_url)) > 5:
        md += "\n[Download slides here](" + item.slides_url + ")\n" 

    if len(str(item.paper_url)) > 5:
        md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    md += "\nRecommended citation: " + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [13]:
!ls ../_publications/

2023-08-18-applicability-of-clay-organic-clay-to-environmental-pollutants-green-way-an-overview.md
2024-02-20-yflows-simd-dataflow-cpu-inference-cc2024.md
2024-09-20-neutral-atom-compiler-decompositions-scheduling-qce2024.md
2025-08-18-tusq-noisy-quantum-simulation-tracking-uncomputation-sampling.md
2025-09-30-q4bio-quantum-enabled-biomarker-discovery-outlook.md


In [14]:
!cat ../_publications/2009-10-01-paper-title-number-1.md

cat: ../_publications/2009-10-01-paper-title-number-1.md: No such file or directory
